In [10]:
from pathlib import Path

import polars as pl

data = pl.read_csv("data/*.csv")

data.groupby(["account", "user_id"]).agg(pl.count()).groupby(["account"]).agg(pl.count())

account,count
str,u32
"""lesRepublicain...",390
"""FranceInsoumis...",711


In [55]:
lr = data.filter(pl.col("account") == "lesRepublicains")
fi = data.filter(pl.col("account") == "FranceInsoumise")

def bow(df):
    return (
        df.select(
            pl.col("tweet")
            .str.to_lowercase()
            .str.split(by=" ").flatten().alias("words")
        )
        .groupby("words")
        .agg(pl.count())
        .sort("count", descending=True)
        .filter((pl.col("words").str.lengths() > 0) & (pl.col("count") > 1))
        .select(
            pl.col("words"),
            (pl.col("count") / pl.col("count").sum()).alias("proportion")
        )
    )


lr_df = bow(lr)
fi_df = bow(fi)
combined = lr_df.join(fi_df, on="words", how="inner", suffix="_FI").rename({"proportion": "proportion_LR"})
combined.filter(pl.col("words").str.starts_with("#"))

words,proportion_LR,proportion_FI
str,f64,f64
"""#""",0.000105,0.000264
"""#nupes""",0.000066,0.000208
"""#reformedesret...",0.000096,0.000203
"""#reformesdesre...",0.000201,0.000194
"""#retraites""",0.000214,0.000133
"""#tiktok""",0.000288,0.000121
"""#nonalareforme...",0.000153,0.000121
"""#manif11fevrie...",0.000162,0.000111
"""#manifestation...",0.000162,0.000087
